In [52]:
import pandas as pd
import plotly.express as px
import re
import datetime
import numpy as np

from Project.Database import Db

# Example data for the TPM

In [53]:
# pd.read_csv("/Users/christiannielsen/Documents/GitHub/Datavidenskab-P6/Project/TPM/Data/ISID.csv")

In [54]:
# pd.read_json(
#     path_or_buf="/Users/christiannielsen/Documents/GitHub/Datavidenskab-P6/Project/TPM/output/Experiment_minsup0.1_minconf_0.1/level1.json",
#     orient='name_node')

In [55]:
year1_minutes, meta = Db.load_data(hourly=False, meta=True, year=1)

In [56]:
meta

,Unnamed: 0,Subsystem,Measurement_Location,Parameter,Description,Units,Aggregation_Method,min_value,max_value
0,DHW_ClothesWasherColdFlow,DHW,Utility,Flow_Water,The cumulative volume of cold water flowing in...,Gallons,Hour value,0.0000,33.0646
1,DHW_ClothesWasherHotFlow,DHW,Utility,Flow_Water,The cumulative volume of hot water flowing int...,Gallons,Hour value,0.0000,201.6969
2,DHW_DishwasherHotFlow,DHW,Kitchen,Flow_Water,The cumulative volume of hot water flowing int...,Gallons,Hour value,0.0000,4.6367
3,DHW_HeatPumpWaterHeaterEnergyTotal,DHW,Basement,Energy_Electrical,Cumulative energy consumption by heat pump wat...,Wh,Hour value,0.0000,188169.7774
4,DHW_HeatPumpWaterHeaterPowerTotal,DHW,Basement,Power_Electrical,Instantaneous power consumption by heat pump w...,W,Average,0.0000,4368.8008
...,...,...,...,...,...,...,...,...,...
350,HVAC_HeatPumpOutdoorUnitPower,HVAC,Outdoor,Power_Electrical,Average power consumption of heat pump outdoor...,W,Average,-68.7461,8624.9120
351,HVAC_DehumidifierAirflow,HVAC,Basement,Flow_Air,Average airflow through dedicated dehumidifier...,ft3/min,Average,NaN,NaN
352,HVAC_DehumidifierExitAirTemp,HVAC,Basement,Temp_Air,Average temperature of air in outlet duct from...,degrees F,Average,NaN,NaN
353,HVAC_DehumidifierInletAirTemp,HVAC,Basement,Temp_Air,Average temperature of air in inlet duct to de...,degrees F,Average,NaN,NaN


In [57]:
appliance_condition = (lambda self: (
        (self["Parameter"] == "Status_OnOff") &
        (self["Subsystem"] == "Loads") &
        (~self.index.str.contains("Child")) &
        (~self.index.str.contains("Prnt")) &
        (~self.index.str.contains("StatusLatentload")) &
        (~self.index.str.contains("Load_StatusClothesWasher")) &
        (~self.index.str.contains("Load_StatusDryerPowerTotal"))
))

appliances_statusOnOff = meta.loc[appliance_condition].index.tolist()

AttributeError: Can only use .str accessor with string values!

In [ ]:
#Replaces values to be binary 0 or 1.
for i in appliances_statusOnOff:
    year1_minutes.loc[year1_minutes[i] < 0, i] = 0
    year1_minutes.loc[year1_minutes[i] > 0, i] = 1

In [ ]:
minute_appliances_status = year1_minutes[["Timestamp"] + appliances_statusOnOff].copy()

In [ ]:
for col in meta["Measurement_Location"].unique():
    appliance_condition = (lambda self: (
            (self["Parameter"] == "Status_OnOff") &
            (self.index.str.contains("Light")) &
            (self["Measurement_Location"] == col)
    ))
    light_cols = meta.loc[appliance_condition].index
    if len(light_cols) > 1:
        minute_appliances_status[col + " Lights"] = minute_appliances_status[light_cols].max(1)
        minute_appliances_status.drop(light_cols, axis= 1, inplace=True)
        for attribute in light_cols:
            appliances_statusOnOff.remove(attribute)
        appliances_statusOnOff.append(col + " Lights")

In [ ]:
#Forces Timestamp to the type of datetime, to extract the hour of Timestamp.
minute_appliances_status["Timestamp"] = pd.to_datetime(minute_appliances_status["Timestamp"],
                                                       format="%Y-%m-%d %H:%M:%S%z", utc=True) - pd.to_timedelta(
    unit="h", arg=(
        minute_appliances_status["Timestamp"].astype("str").str.split(pat="-", expand=True)[lambda self: self.columns[-1]].str[
            1]).astype(
        int))

#Extracts hour of Timestamp.
minute_appliances_status["DayOfYear"] = minute_appliances_status.Timestamp.dt.dayofyear

In [ ]:
minute_appliances_status["DayOfYear"] = minute_appliances_status.Timestamp.dt.dayofyear
minute_appliances_status["DayOfYear"] = minute_appliances_status["DayOfYear"] -  minute_appliances_status["DayOfYear"][0]
minute_appliances_status["DayOfYear"] = np.where(minute_appliances_status["DayOfYear"] < 0, minute_appliances_status["DayOfYear"] + 365, minute_appliances_status["DayOfYear"])
minute_appliances_status.loc[500000:].loc[lambda self: self["DayOfYear"] == 0] = 365
minute_appliances_status

In [ ]:
# minute_appliances_status['Timestamp'] = (pd.to_datetime(minute_appliances_status["Timestamp"],
#                                                        format="%Y-%m-%d %H:%M:%S%z", utc=True).astype("int") / 1e9).astype("int")

In [ ]:
minute_appliances_status['Timestamp'] = (pd.DatetimeIndex ( minute_appliances_status["Timestamp"] ).astype ( np.int64 )/1e9).astype(int)


In [ ]:
minute_appliances_status["DayOfYear"] = ((minute_appliances_status['Timestamp'] - minute_appliances_status['Timestamp'].min()) / (3600 * 24)).astype("int")

In [ ]:
minute_appliances_status

In [ ]:
appliance_job_list = []

for appliance in appliances_statusOnOff:
    name = " ".join(re.findall('[A-Z][^A-Z]*', appliance.split("Load_Status")[-1]))
    appliance_switch = minute_appliances_status[(minute_appliances_status[appliance] != minute_appliances_status[appliance].shift(1))][["Timestamp", "DayOfYear", appliance]][1:]

    df_indices = appliance_switch.index.tolist()
    for index, df_index in enumerate(df_indices):
        if appliance_switch.loc[df_index, appliance]:
            try:
                appliance_job_list.append({"start": appliance_switch.loc[df_index, "Timestamp"],
                                           "end": appliance_switch.loc[df_indices[index + 1], "Timestamp"],
                                           "appliance": name,
                                           "day": appliance_switch.loc[df_indices[index], "DayOfYear"]
                                           })
            except:
                continue

appliance_job_list

In [ ]:
df = pd.DataFrame(appliance_job_list).sort_values(by=['day', 'start']).reset_index(drop=True)
df

In [ ]:
df.to_csv(path_or_buf=Db.get_project_path("Project/04TPMAlgorithm/TPM/Data/houseDataWDryerAndWasher.csv"), header=False, index=False)

In [ ]:
df1 = pd.read_csv(Db.get_project_path("Project/04TPMAlgorithm/TPM/Data/houseData.csv"), header=None)
df2 = pd.read_csv(Db.get_project_path("Project/04TPMAlgorithm/TPM/Data/houseDataWDryerAndWasher.csv"), header=None)

In [ ]:
df1

In [ ]:
df2

In [ ]:
df_diff = pd.concat([df1,df2]).drop_duplicates(keep=False, ignore_index=True)

In [ ]:
df_diff